# Gensim Doc2Vec Tutorial on the Lee Dataset
This notebook is modified based on the original one provided by Gensim.  
You can find it here: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb

In [1]:
import gensim
import os
import collections
import smart_open
import random

## What is it?

Doc2Vec is an NLP tool for representing documents as a vector and is **a generalizing of the Word2Vec method**. This tutorial will serve as an introduction to Doc2Vec and present ways to train and assess a Doc2Vec model.

## Resources

* [Word2Vec Paper](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)
* [Doc2Vec Paper](https://cs.stanford.edu/~quocle/paragraph_vector.pdf)
* [Dr. Michael D. Lee's Website](http://faculty.sites.uci.edu/mdlee)
* [Lee Corpus](http://faculty.sites.uci.edu/mdlee/similarity-data/)
* [IMDB Doc2Vec Tutorial](doc2vec-IMDB.ipynb)

## Getting Started

To get going, we'll need to have **a set of documents** to train our doc2vec model.  
In theory, a document could be anything from a short 140 character tweet, a single paragraph (i.e., journal article abstract), a news article, or a book. In NLP parlance a collection or set of documents is often referred to as a **corpus**.   
  
For this tutorial, we'll be training our model using the [Lee Background Corpus](https://hekyll.services.adelaide.edu.au/dspace/bitstream/2440/28910/1/hdl_28910.pdf) included in gensim. This corpus contains **314 documents** selected from the Australian Broadcasting Corporation’s news mail service, which provides text e-mails of headline stories and covers a number of broad topics.  
  
And we'll **test our model by eye** using the much shorter [Lee Corpus](https://hekyll.services.adelaide.edu.au/dspace/bitstream/2440/28910/1/hdl_28910.pdf) which contains 50 documents.

In [2]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
lee_test_file = test_data_dir + os.sep + 'lee.cor'

## Define a Function to Read and Preprocess Text

Below, we define a function to:  
  
1. open the train/test file (with latin encoding) 
2. read the file line-by-line
3. pre-process each line using a simple gensim pre-processing tool (i.e., tokenize text into individual words, remove punctuation, set to lowercase, etc)
4. return a list of words. 
  
Note that, for a given file (aka corpus), each continuous line constitutes a single document and the length of each line (i.e., document) can vary. Also, to train the model, we'll need to associate a tag/number with each document of the training corpus.  
**In our case, the tag is simply the zero-based line number.**

In [3]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # for training datta, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [4]:
train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

Let's take a look at the **training corpus** (with tags)

In [5]:
train_corpus[:2]

[TaggedDocument(words=[u'hundreds', u'of', u'people', u'have', u'been', u'forced', u'to', u'vacate', u'their', u'homes', u'in', u'the', u'southern', u'highlands', u'of', u'new', u'south', u'wales', u'as', u'strong', u'winds', u'today', u'pushed', u'huge', u'bushfire', u'towards', u'the', u'town', u'of', u'hill', u'top', u'new', u'blaze', u'near', u'goulburn', u'south', u'west', u'of', u'sydney', u'has', u'forced', u'the', u'closure', u'of', u'the', u'hume', u'highway', u'at', u'about', u'pm', u'aedt', u'marked', u'deterioration', u'in', u'the', u'weather', u'as', u'storm', u'cell', u'moved', u'east', u'across', u'the', u'blue', u'mountains', u'forced', u'authorities', u'to', u'make', u'decision', u'to', u'evacuate', u'people', u'from', u'homes', u'in', u'outlying', u'streets', u'at', u'hill', u'top', u'in', u'the', u'new', u'south', u'wales', u'southern', u'highlands', u'an', u'estimated', u'residents', u'have', u'left', u'their', u'homes', u'for', u'nearby', u'mittagong', u'the', u'ne

And the **testing corpus** looks like this (without tags)

In [6]:
print(test_corpus[:2])

[[u'the', u'national', u'executive', u'of', u'the', u'strife', u'torn', u'democrats', u'last', u'night', u'appointed', u'little', u'known', u'west', u'australian', u'senator', u'brian', u'greig', u'as', u'interim', u'leader', u'shock', u'move', u'likely', u'to', u'provoke', u'further', u'conflict', u'between', u'the', u'party', u'senators', u'and', u'its', u'organisation', u'in', u'move', u'to', u'reassert', u'control', u'over', u'the', u'party', u'seven', u'senators', u'the', u'national', u'executive', u'last', u'night', u'rejected', u'aden', u'ridgeway', u'bid', u'to', u'become', u'interim', u'leader', u'in', u'favour', u'of', u'senator', u'greig', u'supporter', u'of', u'deposed', u'leader', u'natasha', u'stott', u'despoja', u'and', u'an', u'outspoken', u'gay', u'rights', u'activist'], [u'cash', u'strapped', u'financial', u'services', u'group', u'amp', u'has', u'shelved', u'million', u'plan', u'to', u'buy', u'shares', u'back', u'from', u'investors', u'and', u'will', u'raise', u'milli

**Notice that the testing corpus is just a list of lists and does not contain any tags.**

## Training the Model

### Instantiate a Doc2Vec Object 

Now, we'll instantiate a Doc2Vec model with a **vector size with 50** and iterating over the training corpus **55 times**.  
We set the minimum word count to 2 in order to give higher frequency words more weighting.  
Model accuracy can be improved by increasing **the number of iterations** but this generally increases the training time. Small datasets with short documents, like this one, can benefit from more training passes.

In [7]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=55)

### Build a Vocabulary

In [8]:
model.build_vocab(train_corpus)

Essentially, the vocabulary is a dictionary (accessible via `model.wv.vocab`) of all of the unique words extracted from the training corpus along with the count (e.g., `model.wv.vocab['penalty'].count` for counts for the word `penalty`).

### Time to Train

If the BLAS library is being used, this should take no more than 3 seconds.
If the BLAS library is not being used, this should take no more than 2 minutes, so use BLAS if you value your time.

In [11]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

CPU times: user 3.82 s, sys: 72 ms, total: 3.89 s
Wall time: 1.53 s


2349197

### Inferring a Vector

One important thing to note is that you can now **infer a vector for any piece of text** without having to re-train the model by passing a list of words to the `model.infer_vector` function. This vector can then be compared with other vectors via cosine similarity.

In [12]:
model.infer_vector(['only', 'you', 'can', 'prevent', 'forrest', 'fires'])

array([ 0.09504433, -0.04072575,  0.05904232, -0.01369987,  0.02581146,
        0.01259436,  0.09162549, -0.09830466, -0.08710306, -0.03994137,
        0.08534875, -0.10230432, -0.01105929, -0.01360191, -0.00642366,
       -0.02888539,  0.038064  ,  0.04591428,  0.05680484, -0.01619486,
       -0.06495674, -0.01177782,  0.08664298,  0.02189884,  0.07222942,
        0.01427946, -0.03322181, -0.05324052,  0.0501392 , -0.03715464,
       -0.03652851,  0.10368759, -0.04504682, -0.02027506,  0.0656474 ,
       -0.10378831, -0.07538308, -0.09834334,  0.1018642 , -0.06898234,
        0.00646124,  0.08503476, -0.07601625, -0.08827069,  0.01583957,
       -0.02621939,  0.0576924 ,  0.03666697,  0.02051398, -0.02968657], dtype=float32)

## Assessing Model

To assess our new model:  
  
1. we'll first infer new vectors for each document of the training corpus
2. compare the inferred vectors with the training corpus
3. then returning the rank of the document based on self-similarity  
  
Basically, we're pretending as if the training corpus is some new unseen data and then seeing how they compare with the trained model.  
The expectation is that we've likely overfit our model (i.e., all of the ranks will be less than 2) and so we should be able to find similar documents very easily.   Additionally, we'll keep track of the second ranks for a comparison of less similar documents.  

In [13]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

Let's count how each document ranks with respect to the training corpus 

In [14]:
# Results vary due to random seeding and very small corpus
collections.Counter(ranks)  

Counter({0: 293, 1: 7})

Basically, greater than **95%** of the inferred documents are found to be most similar to itself and about 5% of the time it is mistakenly most similar to another document.  
The checking of an inferred-vector against a training-vector is a sort of **'sanity check'** as to whether the model is behaving in a usefully consistent manner, though not a real 'accuracy' value.  
This is great and not entirely surprising. We can take a look at an example:

In [15]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (299): «australia will take on france in the doubles rubber of the davis cup tennis final today with the tie levelled at wayne arthurs and todd woodbridge are scheduled to lead australia in the doubles against cedric pioline and fabrice santoro however changes can be made to the line up up to an hour before the match and australian team captain john fitzgerald suggested he might do just that we ll make team appraisal of the whole situation go over the pros and cons and make decision french team captain guy forget says he will not make changes but does not know what to expect from australia todd is the best doubles player in the world right now so expect him to play he said would probably use wayne arthurs but don know what to expect really pat rafter salvaged australia davis cup campaign yesterday with win in the second singles match rafter overcame an arm injury to defeat french number one sebastien grosjean in three sets the australian says he is happy with his form it not v

Notice above that the most similar document is has a similarity score of ~80% (or higher).  
However, the similarity score for the second ranked documents should be significantly lower (assuming the documents are in fact different) and the reasoning becomes obvious when we examine the text itself.  

In [16]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(train_corpus))

# Compare and print the most/median/least similar documents from the train corpus
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))

Train Document (245): «federal treasurer peter costello has warned continued economic growth in australia is dependent on an uncertain world outlook the latest figures show the economy grew by per cent in the september quarter mr costello is stressing the seriousness of the current global economic downturn as serious as anything we ve seen in the last two decades he said both he and the reserve bank governor ian macfarlane believe the timing of any pickup is unclear mr costello is hopeful about the united states prospects next year and says pickup sooner rather than later in the year could keep australia in strong position think with every confidence the december quarter is going to be strong if the us comes back we might defy again what has happened in the world he said shadow treasurer bob mcmullan says the treasurer is left relying on things out of his control all the treasurer has got in plan for is the hope the american economy will pick up in time he said he says the government h

## Testing the Model

Using the same approach above, we'll infer the vector for a randomly chosen test document, and compare the document to our model by eye.

In [17]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus))
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (26): «how did allegedly unregistered missile warheads come to be stored on canadian businessman anti terrorism training facility in new mexico and canadian officials are still trying to figure that out but one security expert says the mystery is chilling one david hudak was arrested in the united states more than week ago when according to court documents agents searching his property found the warheads stored in crates that were marked charge demolition»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

MOST (192, 0.5894472002983093): «the united states is intensifying its bombing of the mountains of eastern afghanistan in concerted attempt to flush out al qaeda fighters the pentagon says more than strike aircraft most of which were fa fighter jets flying from aircraft carriers stationed in the area have flown sorties over afghanistan they were accompanied by long range bombers who targeted areas of the white mountains near jalalabad where osama

### Wrapping Up

That's it! Doc2Vec is a great way to explore relationships between documents.